In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
from IPython.display import clear_output

from skimage import transform
from collections import deque

In [2]:
from vizdoom import DoomGame
from vizdoom import Button
from vizdoom import GameVariable
from vizdoom import ScreenFormat
from vizdoom import ScreenResolution

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [4]:
import warnings
warnings.filterwarnings('ignore')

## Set Configs

In [5]:
is_cuda = torch.cuda.is_available()

if is_cuda: device = torch.device('cuda')
else: device = torch.device('cpu')

## Set Environment

In [6]:
game = DoomGame()

In [7]:
game.load_config("./configs/deadly_corridor.cfg"); game.set_doom_scenario_path("./configs/deadly_corridor.wad");

game.set_screen_resolution(ScreenResolution.RES_640X480)
game.set_screen_format(ScreenFormat.RGB24)
game.set_render_hud(False)
game.set_render_crosshair(False)
game.set_render_weapon(True)
game.set_render_decals(False)
game.set_render_particles(False)
game.add_available_button(Button.MOVE_LEFT)
game.add_available_button(Button.MOVE_RIGHT)
game.add_available_button(Button.ATTACK)
game.add_available_game_variable(GameVariable.AMMO2)
game.set_episode_timeout(200)
game.set_episode_start_time(10)
game.set_window_visible(True)
game.set_living_reward(-1)

In [8]:
game.init()

## Pre-Process Frame

In [9]:
def preprocess_frame(frame):
    
    # crop the screen (remove part that contains no information)
    cropped_frame = frame[15:-5, 20:-20]
    
    # normalize pixel values
    normalized_frame = cropped_frame/ 255.0
    
    # resize the crop frame
    preprocesed_frame = transform.resize(cropped_frame, [100, 120])
    
    return preprocess_frame

In [10]:
stack_size = 4
stacked_frames = deque([np.zeros((100, 200), dtype=np.int) for _ in range(stack_size)], maxlen=4)

In [11]:
def stack_frames(stacked_frames, state, is_new_episode):
    
    # preprocess frame
    frame = preprocess_frame(state)
    
    if is_new_episode:
        
        # clear out the stacked frames
        stacked_frames = deque([np.zeros((100, 120), dtype=np.int) for _ in range(stack_size)], maxlen=4)
        
        # since in a new episode, copy the same frame 4x
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        
        # stack the frames
        stacked_state = np.stack(stacked_frames, axis=2)
        
    else:
        
        # append frame to deque automatically removes the oldest frame
        stacked_frames.append(frame)
        
        # build the stacked state (first dimension specifies different frames)
        stacked_state = np.stack(stacked_frames, axis=2)
        
    return stacked_state, stacked_frames

---